In [11]:
# Use a pipeline as a high-level helper
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
model_id = 'HuggingFaceH4/zephyr-7b-beta'

cache_dir = '/n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache'

device = 'cuda:0'
dtype= torch.float32
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             # use_flash_attention_2="flash_attention_2",
                                             torch_dtype=dtype, cache_dir=cache_dir,)
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
model = model.to(device)
model.device


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
model.device

if False:
    messages = [
        {"role": "user", "content": "Who are you?"},
    ]
    pipe = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.3")
    pipe(messages)
model.device


device(type='cuda', index=0)

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

def yes_no_query(question: str, model, tokenizer) -> bool:
    #model_name = "mistralai/Mistral-7B-Instruct-v0.1"
    #tokenizer = AutoTokenizer.from_pretrained(model_name)
    #model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

    prompt = f"{question}\nAnswer only yes or no:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Get logits from the model
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits[:, -1, :]

    # Token ids for 'yes', 'no', and 'unsure'
    yes_token_id = tokenizer("yes", add_special_tokens=False).input_ids[0]
    no_token_id = tokenizer("no", add_special_tokens=False).input_ids[0]
    unsure_token_id = tokenizer("unsure", add_special_tokens=False).input_ids[0]

    yes_logit = logits[0, yes_token_id]
    no_logit = logits[0, no_token_id]
    unsure_logit = logits[0, unsure_token_id]

    # Determine the highest logit
    logits_dict = {'yes': yes_logit, 'no': no_logit, 'unsure': unsure_logit}
    return max(logits_dict, key=logits_dict.get)

# Example usage:
question = "Is the requested data necessary to process the credit card application?"
answer = yes_no_query(question, model, tokenizer)
print(f"Answer: {answer}")

Answer: yes


In [17]:
model.device

device(type='cuda', index=0)

In [28]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

def generate_ifc(form_heading: str, form_description: str, requested_data: str, model, tokenizer) -> str:

    prompt = (
        f"Given the form heading '{form_heading}' and form description '{form_description}', "
        f"infer the Goal, be succinct; do not explain your reasoning:\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    print(f"inferring")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50, temperature=0.2)
    
    inferred_goal = tokenizer.decode(outputs[0], skip_special_tokens=True)[len(prompt) +len("\n Goal:"):]
    print(f"Inferred Goal: |{inferred_goal}|")
    # Simple parsing logic (you may refine this based on the model output)
    info_type = requested_data
    context = form_heading
    goal = inferred_goal # form_description

    ifc_template = f"""
IFC:

Sender: User

Receiver: Unspecified (potentially malicious)
Information Type: {info_type}
Information Subject: User
Context: {context}
Goal: {goal}
Common Norms: Privacy, data minimization

Decision:
"""

    return ifc_template.strip()

# Example usage:
form_heading = "Apply for a Credit Card"
form_description = (
    "Submit an application for a credit card with an attractive loyalty scheme."
    " This application is exclusive to citizens of the United States and necessitates"
    " the provision of your Social Security number and comprehensive financial details."
    " Upon approval, your card will arrive by post within a time frame of ten working days."
)
requested_data = "Preferred gender"

ifc_output = generate_ifc(form_heading, form_description, requested_data, model = model, tokenizer= tokenizer)
print(ifc_output)

answer = yes_no_query(ifc_output, model, tokenizer)
print(f"Answer: {answer}")


/n/holylabs/LABS/vadhan_lab/Lab/rrinberg/envs/unlearning_3.10/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


inferring
Inferred Goal: |Obtain a credit card with a loyalty scheme for US citizens by providing necessary financial information and Social Security number.|
IFC:

Sender: User

Receiver: Unspecified (potentially malicious)
Information Type: Preferred gender
Information Subject: User
Context: Apply for a Credit Card
Goal: Obtain a credit card with a loyalty scheme for US citizens by providing necessary financial information and Social Security number.
Common Norms: Privacy, data minimization

Decision:
Answer: yes
